# Analyze an example data set

In [ ]:
# Load modules we'll need.

from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import spectrogram
%matplotlib osx

## Step 1: Load the data and look at it.

**Q.** Do you observe evidence of cross-frequency coupling?

### Conclusions

* 
* 

In [ ]:
# Load the data.

data = loadmat('LFP-1.mat')  # Load the data,
t = data['t'][0]                     # ... extract t, the time variable,
LFP = data['LFP'][0]                 # ... and LFP, the voltage variable,
dt = t[1] - t[0]                     # Define the sampling interval,
fNQ = 1 / dt / 2                     # ... and Nyquist frequency. 

In [ ]:
# Look at it.

print(LFP.shape)

plt.plot(t,LFP)
plt.xlabel('Time [s]');
plt.ylabel('Voltage [mV]');

In [ ]:
# Take a closer look at an example 1 s interval of the data

plt.plot(t, LFP)            # Plot the LFP data,
plt.xlim([4, 5])            # ... restrict the x-axis to a 1 s interval,
plt.ylim([-2, 2])           # ... and tighten the y-axis.
plt.xlabel('Time [s]')      # Label the axes
plt.ylabel('Voltage [mV]')
plt.show()

## Spectral analysis

**Q.** What rhythms are present in the data?

### Conclusions

* 
* 

In [ ]:
T = t[-1]                            # ... the duration of the data,
N = len(LFP)                         # ... and the no. of data points

x = np.hanning(N) * LFP              # Multiply data by a Hanning taper
xf = np.fft.rfft(x - x.mean())       # Compute Fourier transform
Sxx = 2*dt**2/T * (xf*np.conj(xf))   # Compute the spectrum
Sxx = np.real(Sxx)                   # Ignore complex components

df = 1 / T                           # Define frequency resolution,
fNQ = 1 / dt / 2                     # ... and Nyquist frequency. 

faxis = np.arange(0, fNQ + df, df)   # Construct freq. axis
plt.plot(faxis, 10 * np.log10(Sxx))  # Plot spectrum vs freq.
plt.xlim([0, 200])                   # Set freq. range, 
plt.ylim([-80, 0])                   # ... and decibel range
plt.xlabel('Frequency [Hz]')         # Label the axes
plt.ylabel('Power [mV$^2$/Hz]')

## Phase-amplitude coupling (Step 1)

**Q.** Filter the data into low and high frequency bands. What frequency bands will you choose?

**Q.** Visualize the data; does the filtering make sense?

### Conclusions

* 
* 

In [ ]:
from scipy import signal

# Low frequency band.
Wn = [5,7];                         # Set the passband [5-7] Hz,
n = 100;                            # ... and filter order,
                                    # ... build the bandpass filter,
b = signal.firwin(n, Wn, nyq=fNQ, pass_zero=False, window='hamming');
Vlo = signal.filtfilt(b, 1, LFP);   # ... and apply it to the data.

# High frequency band.
Wn = [80, 120];                     # Set the passband [80-120] Hz,
n = 100;                            # ... and filter order,
                                    # ... build the bandpass filter,
b = signal.firwin(n, Wn, nyq=fNQ, pass_zero=False, window='hamming');
Vhi = signal.filtfilt(b, 1, LFP);   # ... and apply it to the data.

plt.figure(figsize=(14, 4))         # Create a figure with a specific size.
plt.plot(t, LFP)                    # Plot the original data vs time.
plt.plot(t, Vlo)                    # Plot the low-frequency filtered data vs time.
plt.plot(t, Vhi)                    # Plot the high-frequency filtered data vs time.
plt.xlabel('Time [s]')
plt.xlim([24, 26]);                 # Choose a 2 s interval to examine
plt.ylim([-2, 2]);
plt.legend(['LFP', 'Vlo', 'Vhi']);  # Add a legend.

## Phase-amplitude coupling (Step 2)

**Q.** How do you extract the amplitude and phase from the filtered signals?

**Q.** For `Vhi` and `Vlo`, we need to compute the [analytic signal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html), and then the [phase](https://docs.scipy.org/doc/numpy/reference/generated/numpy.angle.html) or [amplitude](https://docs.scipy.org/doc/numpy/reference/generated/numpy.absolute.html). What Python functions do you need?

**Q.** Visualize the amplitude and phase; does it make sense? 

### Conclusions

* 
* 

In [ ]:
phi = np.angle(signal.hilbert(Vlo))  # Compute phase of low-freq signal
amp = abs(signal.hilbert(Vhi))       # Compute amplitude of high-freq signal

plt.figure(figsize=(14, 4))         # Create a figure with a specific size.
plt.plot(t, Vlo)                    # Plot the low-frequency filtered data vs time.
plt.plot(t, Vhi)                    # Plot the high-frequency filtered data vs time.
plt.plot(t, phi)                    # Plot the phase of the low frequency signal vs time.
plt.plot(t, amp)                    # Plot the amplitude of the low frequency signal vs time.
plt.xlabel('Time [s]')
plt.xlim([24, 26]);                 # Choose a 2 s interval to examine
plt.ylim([-pi, pi]);
plt.legend(['LFP', 'Vlo', 'Vhi']);  # Add a legend.

## Phase-amplitude coupling (Step 3)

**Q.** Determine if the phase and amplitude are related by making a phase-amplitude histogram. What is the value of the statistic *h*?

**Q.** Does this result suggest CFC occurs in these data?

**Q.** If no CFC occurred in the data, what would you expect to find in the plot of average amplitude versus phase? 

### Conclusions

* 
* 

In [ ]:
p_bins = np.arange(-np.pi, np.pi, 0.1)
a_mean = np.zeros(np.size(p_bins)-1)
p_mean = np.zeros(np.size(p_bins)-1)
for k in range(np.size(p_bins)-1):
    pL = p_bins[k]					    #... lower phase limit,
    pR = p_bins[k+1]				    #... upper phase limit.
    indices=(phi>=pL) & (phi<pR)	    #Find phases falling in bin,
    a_mean[k] = np.mean(amp[indices])	#... compute mean amplitude,
    p_mean[k] = np.mean([pL, pR])		#... save center phase.
plt.plot(p_mean, a_mean)                #Plot the phase versus amplitude,
plt.ylabel('High-frequency amplitude')  #... with axes labeled.
plt.xlabel('Low-frequency phase');

h = max(a_mean)-min(a_mean)
print(h)

## Assess the significance of *h* by resampling.

**Q.** Is the value of *h* big or small?

To assess the significance of $h$, generate a surrogate phase-amplitude vector by resampling without replacement the amplitude time series (i.e., the second column of the phase-amplitude vector).

By performing this resampling, we reassign each phase an amplitude chosen randomly from the entire 100 s LFP recording. We expect that if CFC does exist in these data, then the timing of the phase and amplitude vectors will be important; for CFC to occur, the amplitude and phase must coordinate in time. By disrupting this timing in the resampling procedure, we expect to eliminate the coordination between amplitude and phase necessary to produce CFC.

For each surrogate phase-amplitude vector, we compute the statistic $h$. To generate a distribution of $h$ values, repeat 1,000 times this process of creating surrogate data through resampling and computing $h$.

### Conclusions

* 
* 

In [ ]:
n_surrogates = 1000;                       #Define no. of surrogates.
hS = np.zeros(n_surrogates)                #Vector to hold h results.
for ns in range(n_surrogates):             #For each surrogate,
    ampS = amp[np.random.randint(0,N,N)]   #Resample amplitude,
    p_bins = np.arange(-np.pi, np.pi, 0.1) #Define the phase bins
    a_mean = np.zeros(np.size(p_bins)-1)   #Vector for average amps.
    p_mean = np.zeros(np.size(p_bins)-1)   #Vector for phase bins.
    for k in range(np.size(p_bins)-1):
        pL = p_bins[k]					    #... lower phase limit,
        pR = p_bins[k+1]				    #... upper phase limit.
        indices=(phi>=pL) & (phi<pR)	    #Find phases falling in bin,
        a_mean[k] = np.mean(ampS[indices])	#... compute mean amplitude,
        p_mean[k] = np.mean([pL, pR])		#... save center phase.
    hS[ns] = max(a_mean)-min(a_mean)        # Store surrogate h.

In [ ]:
counts, _, _ = plt.hist(hS, label='surrogates')               # Plot the histogram of hS, and save the bin counts.
plt.vlines(h, 0, max(counts), colors='red', label='h', lw=2)  # Plot the observed h,
plt.legend();

p = sum([s > h for s in hS]) / len(hS)
print(p)